# Importação e preparação dos dados

In [1]:
import pandas as pd

df = pd.read_csv('../dados/jurisprudencia_selecionada_excertos.CSV', sep = ';')
df.head()

,COD,NUM_ENUNCIADO,COD_AREA,DESCR_AREA,COD_TEMA,DESCR_TEMA,COD_SUBTEMA,DESCR_SUBTEMA,COD_DOC_TRAMITAVEL_EXCERTO,TEXTO_EXCERTO,ACORDAO,TIPO_PROCESSO
0,1400,1236,50,Responsabilidade,488,Solidariedade,261,Benefício previdenciário,54995438,Voto:Cuidam os autos de tomada de contas espec...,Acórdão 297/2016 - PL,Tomada de Contas Especial
1,1700,1534,46,Finanças Públicas,981,Exportação,983,Petróleo,55025603,Voto:Cuidam os autos de Solicitação do Congres...,Acórdão 366/2016 - PL,Solicitação do Congresso Nacional
2,5700,5314,50,Responsabilidade,203,Multa,1021,Dosimetria,55455375,Relatório:Trata-se de embargos de declaração o...,Acórdão 944/2016 - PL,Acompanhamento
3,284,40,45,Direito Processual,162,Princípio da independência das instâncias,481,Decisão judicial,54773747,Voto:8. Em relação a outros processos judiciai...,Acórdão 30/2016 - PL,Tomada de Contas Especial
4,298,54,49,Pessoal,141,Sistema S,142,Nepotismo,54773403,Voto:11. Relativamente ao ato envolvendo a Sra...,Acórdão 55/2016 - PL,Representação


In [2]:
df.shape

(13285, 12)

In [3]:
df.groupby(['DESCR_AREA']).size()

DESCR_AREA
Competência do TCU          553
Contrato Administrativo     941
Convênio                    683
Desestatização              139
Direito Processual         1811
Finanças Públicas           328
Gestão Administrativa       338
Licitação                  2756
Pessoal                    3393
Responsabilidade           2343
dtype: int64

In [4]:
areas = df.groupby(['DESCR_AREA']).groups.keys()
areas

dict_keys(['Competência do TCU', 'Contrato Administrativo', 'Convênio', 'Desestatização', 'Direito Processual', 'Finanças Públicas', 'Gestão Administrativa', 'Licitação', 'Pessoal', 'Responsabilidade'])

In [5]:
from sklearn.preprocessing import LabelBinarizer

lbArea = LabelBinarizer()
lbArea.fit([x for x in areas])
lbArea.classes_

array(['Competência do TCU', 'Contrato Administrativo', 'Convênio',
       'Desestatização', 'Direito Processual', 'Finanças Públicas',
       'Gestão Administrativa', 'Licitação', 'Pessoal',
       'Responsabilidade'], dtype='<U23')

In [6]:
y = lbArea.transform(df['DESCR_AREA'])
y.shape

(13285, 10)

In [7]:
from keras.preprocessing.text import Tokenizer
import numpy as np

limite_texto = 2000
dim_vetor = 50

tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['TEXTO_EXCERTO'])

word_index = tokenizer.word_index
vocabulario = len(word_index) + 1
print('Found %s unique tokens.' % len(word_index))

sequences = tokenizer.texts_to_sequences(df['TEXTO_EXCERTO'])

Using TensorFlow backend.


Found 63925 unique tokens.


In [8]:
from keras.preprocessing.sequence import pad_sequences

x = pad_sequences(sequences, maxlen=limite_texto)

print('Shape of data tensor:', x.shape)

Shape of data tensor: (13285, 2000)


In [9]:
from gensim.models import Word2Vec

model = Word2Vec.load('../vocabularios/modelo-acordaos-50.w2v')

/home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [10]:
# create a weight matrix for words in training docs

embedding_matrix = np.zeros((vocabulario, dim_vetor))

ok = 0
for word, i in tokenizer.word_index.items():
    if word in model.wv:
        embedding_matrix[i] = model.wv[word]
        ok += 1
print('Vocabulario:', i)
print('Encontrados no modelo:', ok, '=', ok * 100. / i)

Vocabulario: 63925
Encontrados no modelo: 43317 = 67.76222135314822


# Treinamento

In [13]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, GRU
from keras.optimizers import RMSprop

model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=False))
model.add(GRU(256))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

W1117 10:51:56.173063 140664023926592 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1117 10:51:56.232472 140664023926592 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1117 10:51:56.234180 140664023926592 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1117 10:51:56.242867 140664023926592 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprec

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 2000, 50)          3196300   
_________________________________________________________________
gru_1 (GRU)                  (None, 256)               235776    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                2570      
Total params: 3,434,646
Trainable params: 238,346
Non-trainable params: 3,196,300
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 496s 47ms/step - loss: 1.1887 - categorical_accuracy: 0.6107 - val_loss: 1.5734 - val_categorical_accuracy: 0.4275
Epoch 2/20
10628/10628 [==============================] - 494s 47ms/step - loss: 0.6477 - categorical_accuracy: 0.7839 - val_loss: 1.0583 - val_categorical_accuracy: 0.

In [14]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(GRU(256))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 2000, 50)          3196300   
_________________________________________________________________
gru_2 (GRU)                  (None, 256)               235776    
_________________________________________________________________
dense_2 (Dense)              (None, 10)                2570      
Total params: 3,434,646
Trainable params: 3,434,646
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 539s 51ms/step - loss: 1.1724 - categorical_accuracy: 0.6154 - val_loss: 1.3386 - val_categorical_accuracy: 0.5292
Epoch 2/20
10628/10628 [==============================] - 539s 51ms/step - loss: 0.6532 - categorical_accuracy: 0.7811 - val_loss: 0.9189 - val_categorical_accuracy: 0.6887
E

In [15]:
from keras.layers import LSTM

model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(LSTM(256))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 2000, 50)          3196300   
_________________________________________________________________
lstm_1 (LSTM)                (None, 256)               314368    
_________________________________________________________________
dense_3 (Dense)              (None, 10)                2570      
Total params: 3,513,238
Trainable params: 3,513,238
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 664s 63ms/step - loss: 1.2672 - categorical_accuracy: 0.5850 - val_loss: 1.2962 - val_categorical_accuracy: 0.5551
Epoch 2/20
10628/10628 [==============================] - 663s 62ms/step - loss: 0.9211 - categorical_accuracy: 0.6916 - val_loss: 1.1158 - val_categorical_accuracy: 0.6229
E

In [18]:
from keras.layers import Bidirectional

model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(Bidirectional(GRU(256)))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 2000, 50)          3196300   
_________________________________________________________________
bidirectional_2 (Bidirection (None, 512)               471552    
_________________________________________________________________
dense_5 (Dense)              (None, 10)                5130      
Total params: 3,672,982
Trainable params: 3,672,982
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 1069s 101ms/step - loss: 1.1831 - categorical_accuracy: 0.6103 - val_loss: 1.0216 - val_categorical_accuracy: 0.6703
Epoch 2/20
10628/10628 [==============================] - 1127s 106ms/step - loss: 0.6487 - categorical_accuracy: 0.7855 - val_loss: 0.9203 - val_categorical_accuracy: 0.71

In [19]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(GRU(128))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 2000, 50)          3196300   
_________________________________________________________________
gru_5 (GRU)                  (None, 128)               68736     
_________________________________________________________________
dense_6 (Dense)              (None, 10)                1290      
Total params: 3,266,326
Trainable params: 3,266,326
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 545s 51ms/step - loss: 1.3138 - categorical_accuracy: 0.5663 - val_loss: 1.5275 - val_categorical_accuracy: 0.5250
Epoch 2/20
10628/10628 [==============================] - 544s 51ms/step - loss: 0.8179 - categorical_accuracy: 0.7260 - val_loss: 1.0089 - val_categorical_accuracy: 0.6590
E

In [13]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(GRU(64))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 2000, 50)          3196300   
_________________________________________________________________
gru_2 (GRU)                  (None, 64)                22080     
_________________________________________________________________
dense_2 (Dense)              (None, 10)                650       
Total params: 3,219,030
Trainable params: 3,219,030
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 557s 52ms/step - loss: 1.5150 - categorical_accuracy: 0.4932 - val_loss: 1.4853 - val_categorical_accuracy: 0.4953
Epoch 2/20
10628/10628 [==============================] - 552s 52ms/step - loss: 0.9883 - categorical_accuracy: 0.6738 - val_loss: 1.2391 - val_categorical_accuracy: 0.5800
E

In [14]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(GRU(512))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 2000, 50)          3196300   
_________________________________________________________________
gru_3 (GRU)                  (None, 512)               864768    
_________________________________________________________________
dense_3 (Dense)              (None, 10)                5130      
Total params: 4,066,198
Trainable params: 4,066,198
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 538s 51ms/step - loss: 1.1404 - categorical_accuracy: 0.6228 - val_loss: 1.3085 - val_categorical_accuracy: 0.5269
Epoch 2/20
10628/10628 [==============================] - 538s 51ms/step - loss: 0.5759 - categorical_accuracy: 0.8060 - val_loss: 0.8210 - val_categorical_accuracy: 0.7335
E

In [15]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(GRU(256))
model.add(Dense(128, activation='relu'))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 2000, 50)          3196300   
_________________________________________________________________
gru_4 (GRU)                  (None, 256)               235776    
_________________________________________________________________
dense_4 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_5 (Dense)              (None, 10)                1290      
Total params: 3,466,262
Trainable params: 3,466,262
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 558s 52ms/step - loss: 1.1445 - categorical_accuracy: 0.6195 - val_loss: 1.1129 - val_categorical_accuracy: 0.6123
Epoch 2/20
10628/10628 [==================

In [16]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(GRU(256))
model.add(Dense(32, activation='relu'))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 2000, 50)          3196300   
_________________________________________________________________
gru_5 (GRU)                  (None, 256)               235776    
_________________________________________________________________
dense_6 (Dense)              (None, 32)                8224      
_________________________________________________________________
dense_7 (Dense)              (None, 10)                330       
Total params: 3,440,630
Trainable params: 3,440,630
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 550s 52ms/step - loss: 1.1956 - categorical_accuracy: 0.6055 - val_loss: 1.3316 - val_categorical_accuracy: 0.6101
Epoch 2/20
10628/10628 [==================

In [17]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(GRU(256, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

W1118 23:00:14.174243 140056940074816 deprecation.py:506] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 2000, 50)          3196300   
_________________________________________________________________
gru_6 (GRU)                  (None, 256)               235776    
_________________________________________________________________
dense_8 (Dense)              (None, 10)                2570      
Total params: 3,434,646
Trainable params: 3,434,646
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 723s 68ms/step - loss: 1.3133 - categorical_accuracy: 0.5693 - val_loss: 1.2415 - val_categorical_accuracy: 0.6044
Epoch 2/20
10628/10628 [==============================] - 697s 66ms/step - loss: 0.8002 - categorical_accuracy: 0.7318 - val_loss: 0.8695 - val_categorical_accuracy: 0.7140
E

In [18]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(GRU(256, dropout=0.1, recurrent_dropout=0.5))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 2000, 50)          3196300   
_________________________________________________________________
gru_7 (GRU)                  (None, 256)               235776    
_________________________________________________________________
dense_9 (Dense)              (None, 10)                2570      
Total params: 3,434,646
Trainable params: 3,434,646
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 655s 62ms/step - loss: 1.3585 - categorical_accuracy: 0.5479 - val_loss: 1.2430 - val_categorical_accuracy: 0.5563
Epoch 2/20
10628/10628 [==============================] - 654s 62ms/step - loss: 0.8358 - categorical_accuracy: 0.7209 - val_loss: 1.0945 - val_categorical_accuracy: 0.6406
E

In [19]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(GRU(256))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam',  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 2000, 50)          3196300   
_________________________________________________________________
gru_8 (GRU)                  (None, 256)               235776    
_________________________________________________________________
dense_10 (Dense)             (None, 10)                2570      
Total params: 3,434,646
Trainable params: 3,434,646
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 542s 51ms/step - loss: 1.1731 - categorical_accuracy: 0.6049 - val_loss: 1.0362 - val_categorical_accuracy: 0.6462
Epoch 2/20
10628/10628 [==============================] - 541s 51ms/step - loss: 0.6309 - categorical_accuracy: 0.7861 - val_loss: 0.8618 - val_categorical_accuracy: 0.7155
E

In [20]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(GRU(256))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adadelta',  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, 2000, 50)          3196300   
_________________________________________________________________
gru_9 (GRU)                  (None, 256)               235776    
_________________________________________________________________
dense_11 (Dense)             (None, 10)                2570      
Total params: 3,434,646
Trainable params: 3,434,646
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 557s 52ms/step - loss: 1.2083 - categorical_accuracy: 0.5990 - val_loss: 2.1070 - val_categorical_accuracy: 0.3459
Epoch 2/20
10628/10628 [==============================] - 556s 52ms/step - loss: 0.6827 - categorical_accuracy: 0.7703 - val_loss: 1.0949 - val_categorical_accuracy: 0.5875
E

In [21]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(GRU(256, return_sequences=True))
model.add(GRU(32))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (None, 2000, 50)          3196300   
_________________________________________________________________
gru_10 (GRU)                 (None, 2000, 256)         235776    
_________________________________________________________________
gru_11 (GRU)                 (None, 32)                27744     
_________________________________________________________________
dense_12 (Dense)             (None, 10)                330       
Total params: 3,460,150
Trainable params: 3,460,150
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 1138s 107ms/step - loss: 1.0759 - categorical_accuracy: 0.6505 - val_loss: 0.9702 - val_categorical_accuracy: 0.6899
Epoch 2/20
10628/10628 [================

In [22]:
from keras.layers.core import Dropout

model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(GRU(256, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_11 (Embedding)     (None, 2000, 50)          3196300   
_________________________________________________________________
gru_12 (GRU)                 (None, 256)               235776    
_________________________________________________________________
dense_13 (Dense)             (None, 32)                8224      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_14 (Dense)             (None, 10)                330       
Total params: 3,440,630
Trainable params: 3,440,630
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 660s 62ms/step - loss: 1.

In [23]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(GRU(256, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_12 (Embedding)     (None, 2000, 50)          3196300   
_________________________________________________________________
gru_13 (GRU)                 (None, 256)               235776    
_________________________________________________________________
dense_15 (Dense)             (None, 10)                2570      
Total params: 3,434,646
Trainable params: 3,434,646
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 668s 63ms/step - loss: 1.3136 - categorical_accuracy: 0.5665 - val_loss: 1.0886 - val_categorical_accuracy: 0.6534
Epoch 2/20
10628/10628 [==============================] - 665s 63ms/step - loss: 0.8006 - categorical_accuracy: 0.7346 - val_loss: 0.9265 - val_categorical_accuracy: 0.7076
E

In [24]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(Bidirectional(GRU(256, dropout=0.2, recurrent_dropout=0.2)))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_13 (Embedding)     (None, 2000, 50)          3196300   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 512)               471552    
_________________________________________________________________
dense_16 (Dense)             (None, 10)                5130      
Total params: 3,672,982
Trainable params: 3,672,982
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 1276s 120ms/step - loss: 1.2666 - categorical_accuracy: 0.5802 - val_loss: 1.5014 - val_categorical_accuracy: 0.5284
Epoch 2/20
10628/10628 [==============================] - 1271s 120ms/step - loss: 0.7891 - categorical_accuracy: 0.7352 - val_loss: 1.0148 - val_categorical_accuracy: 0.68

In [25]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(GRU(512, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_14 (Embedding)     (None, 2000, 50)          3196300   
_________________________________________________________________
gru_15 (GRU)                 (None, 512)               864768    
_________________________________________________________________
dense_17 (Dense)             (None, 10)                5130      
Total params: 4,066,198
Trainable params: 4,066,198
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 662s 62ms/step - loss: 1.2411 - categorical_accuracy: 0.5889 - val_loss: 1.1782 - val_categorical_accuracy: 0.5939
Epoch 2/20
10628/10628 [==============================] - 660s 62ms/step - loss: 0.7062 - categorical_accuracy: 0.7609 - val_loss: 0.8605 - val_categorical_accuracy: 0.7094
E

In [26]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(GRU(1024, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_15 (Embedding)     (None, 2000, 50)          3196300   
_________________________________________________________________
gru_16 (GRU)                 (None, 1024)              3302400   
_________________________________________________________________
dense_18 (Dense)             (None, 10)                10250     
Total params: 6,508,950
Trainable params: 6,508,950
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 706s 66ms/step - loss: 1.3598 - categorical_accuracy: 0.5544 - val_loss: 1.2007 - val_categorical_accuracy: 0.5709
Epoch 2/20
10628/10628 [==============================] - 704s 66ms/step - loss: 0.8078 - categorical_accuracy: 0.7269 - val_loss: 1.2410 - val_categorical_accuracy: 0.5548
E

In [27]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(LSTM(512, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_16 (Embedding)     (None, 2000, 50)          3196300   
_________________________________________________________________
lstm_1 (LSTM)                (None, 512)               1153024   
_________________________________________________________________
dense_19 (Dense)             (None, 10)                5130      
Total params: 4,354,454
Trainable params: 4,354,454
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 822s 77ms/step - loss: 1.3392 - categorical_accuracy: 0.5648 - val_loss: 1.6619 - val_categorical_accuracy: 0.4829
Epoch 2/20
10628/10628 [==============================] - 820s 77ms/step - loss: 0.9661 - categorical_accuracy: 0.6795 - val_loss: 1.5405 - val_categorical_accuracy: 0.4930
E

In [28]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(LSTM(1024, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_17 (Embedding)     (None, 2000, 50)          3196300   
_________________________________________________________________
lstm_2 (LSTM)                (None, 1024)              4403200   
_________________________________________________________________
dense_20 (Dense)             (None, 10)                10250     
Total params: 7,609,750
Trainable params: 7,609,750
Non-trainable params: 0
_________________________________________________________________
Train on 10628 samples, validate on 2657 samples
Epoch 1/20
10628/10628 [==============================] - 946s 89ms/step - loss: 1.3551 - categorical_accuracy: 0.5589 - val_loss: 1.3461 - val_categorical_accuracy: 0.5597
Epoch 2/20
10628/10628 [==============================] - 943s 89ms/step - loss: 0.9785 - categorical_accuracy: 0.6816 - val_loss: 1.1574 - val_categorical_accuracy: 0.5747
E